## ¿Qué hace `features.py`?

El script `features.py` es el **núcleo de la ingeniería de características** para nuestro modelo de predicción de valor de cliente (LTV), costo de adquisición (CAC) y cohortes. Transforma los datos crudos de visitas, órdenes y costos en un **dataset enriquecido y listo para el modelado**.

---

### Funciones principales del script

---

### 1. `get_behavioral_features(visits, orders)`
Extrae **características de comportamiento del usuario**:
- `n_sessions`: número total de sesiones.
- `avg_session_duration`: duración promedio de las sesiones (en minutos).
- `session_duration_std`: desviación estándar de duración de sesiones (consistencia).
- `n_orders`, `revenue_total`, `avg_order_value`: volumen y valor de compras.
- `conversion_delay_days`: días desde la primera visita hasta la primera compra.
- `order_span_days`: días entre la primera y la última compra.
- `orders_per_session`: eficiencia del usuario (órdenes por sesión).
- `avg_days_between_orders`: frecuencia promedio entre pedidos.
- `is_churned`: marca si el usuario no regresó después de 30 días.

---

### 2. `get_temporal_features(visits, orders)`
Genera **features temporales enriquecidas**:
- `session_month`, `session_day`, `session_quarter`: ubicación en el calendario.
- `session_weekday`, `session_hour`: día y hora de primera sesión.
- `is_weekend_session`: indica si la sesión ocurrió fin de semana.
- `conversion_weekday`: día de la semana de la conversión (compra).

---

### 3. `get_marketing_features(visits, costs, orders)`
Crea **variables derivadas del canal de marketing**:
- `device`, `source_id`: canal y medio desde el que llegó el usuario.
- `source_conversion_rate`: porcentaje de conversión por fuente (eficacia).
- `avg_cost_per_user`: gasto de marketing promedio por usuario por fuente.

---

### 4. `generate_ltv_180(visits, orders)`
Calcula el target **LTV_180**:
- Ingresos totales por usuario en los 180 días posteriores a su primera sesión.

---

### 5. `generate_cac_source_30(visits, orders, costs)`
Calcula el target **CAC_source_30**:
- Costos asociados al canal del usuario, medidos en los 30 días después de la primera compra.

---

### 6. `get_cohort_features(visits, ltv, cac, orders)`
Agrega **features agregadas por cohorte mensual**:
- `ltv_cohort_avg`: promedio de LTV en su cohorte de incorporación.
- `cac_cohort_avg`: CAC promedio del mes de incorporación.
- `conversion_rate_cohort`: tasa de conversión dentro de la cohorte.

Esto permite introducir señales temporales agregadas sin sobreajuste.

---

### 7. `generate_feature_dataset(visits, orders, costs)`
Función principal que:
- Llama a todas las funciones anteriores.
- Une todas las features y targets por `uid`.
- Devuelve un **DataFrame final enriquecido** y guardable como CSV (`final_dataset_with_cohort_features.csv`).

---

**Resultado:** El script unifica los tres datasets (`visits`, `orders`, `costs`) en uno solo, que contiene información de comportamiento, calendario, marketing, objetivos de negocio y cohortes. Este dataset es **apto para modelado supervisado de LTV, CAC y ROMI**, con un diseño que **minimiza la colinealidad y reduce el riesgo de overfitting**.


In [1]:
import os
import sys
import pandas as pd

# Añadir la ruta del archivo features.py al path
src_path = os.path.abspath(os.path.join(os.getcwd(), '../src'))
if src_path not in sys.path:
    sys.path.append(src_path)
    
from features import generate_feature_dataset

# Cargar datos limpios
visits = pd.read_csv("../data/processed/visits_clean.csv", parse_dates=['start_ts', 'end_ts'])
orders = pd.read_csv("../data/processed/orders_clean.csv", parse_dates=['buy_ts'])
costs = pd.read_csv("../data/processed/costs_clean.csv", parse_dates=['dt'])

# Generar dataset final
final_dataset = generate_feature_dataset(visits, orders, costs)

# Guardar en processed/
final_dataset.to_csv("../data/processed/final_dataset.csv", index=False)

Generando features()...
Dataset generado. Shape final: (228169, 33)
